In [1]:
import numpy as np
import pandas as pd
import pickle
import os.path
import math
import sys
import csv
import matplotlib.pyplot as plt
from sklearn.externals import joblib
import scipy.integrate as spi
import pylab as pl

In [2]:
n_bytes = 2**31
max_bytes = 2**31 - 1

def load_pickle(file_path):
    bytes_in = bytearray(0)
    input_size = os.path.getsize(file_path)
    with open(file_path, 'rb') as f_in:
        for _ in range(0, input_size, max_bytes):
            bytes_in += f_in.read(max_bytes)
        return pickle.loads(bytes_in)
path = "/Users/maimai/Desktop/project/May/data/givenchy/pickle"  
users = load_pickle(path+"/users.dat")
friends = load_pickle(path+"/friends.dat")
network = load_pickle(path+"/network.dat")
old_data = load_pickle(path+"/30_data(0).dat")
data = load_pickle(path+"/30_data.dat")

In [3]:
old_data.head()

,label,user_index,UsM_deltaDays,UsM_statusesCount,UsM_followersCount,UsM_favouritesCount,UsM_friendsCount,UsM_listedCount,UsM_normalizedUserStatusesCount,UsM_normalizedUserFollowersCount,...,Stat_average_favouritesCount,Stat_average_friendsCount,Stat_average_listedCount,Stat_average_normalizedUserStatusesCount,Stat_average_normalizedUserFollowersCount,Stat_average_normalizedUserFavouritesCount,Stat_average_normalizedUserListedCount,Stat_average_normalizedUserFriendsCount,Stat_max_kOut,Stat_min_kOut
0,1,1,1042,12853,753,34186,3428,8,12.334933,0.722649,...,11938.0,4643.0,555.0,60.605747,47.817908,3.98864,0.185433,1.551286,183,183
1,1,1,1042,12853,753,34186,3428,8,12.334933,0.722649,...,11938.0,4643.0,555.0,60.605747,47.817908,3.98864,0.185433,1.551286,183,183
2,1,1,1042,12853,753,34186,3428,8,12.334933,0.722649,...,11938.0,4643.0,555.0,60.605747,47.817908,3.98864,0.185433,1.551286,183,183
3,1,1,1042,12853,753,34186,3428,8,12.334933,0.722649,...,11938.0,4643.0,555.0,60.605747,47.817908,3.98864,0.185433,1.551286,183,183
4,1,1,1042,12853,753,34186,3428,8,12.334933,0.722649,...,11938.0,4643.0,555.0,60.605747,47.817908,3.98864,0.185433,1.551286,183,183


In [4]:
network.head(3)

,user,user_id,source_id,time_lapsed,followers_count,friends_count,seed_user_id,generation,time_since_seed
0,davelackie,100766356,NaN,0.00,143119,4643,100766356.0,0.0,0.00
1,alexanderskhan,3186545203,100766356.0,0.15,753,3428,100766356.0,1.0,0.15
2,consiglierela,4134992843,100766356.0,0.23,2161,396,100766356.0,1.0,0.23


In [5]:
users.head(3)

,user,user_id,time_lapsed,followers_count,friends_count,user_created_days,user_statuses_count,user_listed_count,user_favourites_count,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed
0,davelackie,100766356,0.00,143119,4643,2993,181393,555,11938,60.605747,47.817908,3.988640,0.185433,1.551286,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0
1,alexanderskhan,3186545203,0.15,753,3428,1042,12853,8,34186,12.334933,0.722649,32.808061,0.007678,3.289827,[0],[0],0,0,1,0.15
2,consiglierela,4134992843,0.23,2161,396,856,46934,36,78797,54.829439,2.524533,92.052570,0.042056,0.462617,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23


In [6]:
######## infec_candidates ########
temp=[]
infec = [temp for i in range(5011)]
users["infec_candidates"]= None
record_num = int(network.describe().iloc[0,0])

for i in range(record_num):
    source_candidates = users.loc[i,"source_candidates"]
    #users.loc[i,"infec_candidates"] = list()
    for j in range(len(source_candidates)):
        candidates = source_candidates[j]
        if infec[candidates] == []:
            infec[candidates]=[i]
        else :
            infec[candidates]+=[i]

users.loc[:,"infec_candidates"]=infec

users.head()

,user,user_id,time_lapsed,followers_count,friends_count,user_created_days,user_statuses_count,user_listed_count,user_favourites_count,normalized_user_statuses_count,...,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed,infec_candidates
0,davelackie,100766356,0.00,143119,4643,2993,181393,555,11938,60.605747,...,3.988640,0.185433,1.551286,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0,"[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1..."
1,alexanderskhan,3186545203,0.15,753,3428,1042,12853,8,34186,12.334933,...,32.808061,0.007678,3.289827,[0],[0],0,0,1,0.15,"[14, 4334]"
2,consiglierela,4134992843,0.23,2161,396,856,46934,36,78797,54.829439,...,92.052570,0.042056,0.462617,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23,"[12, 32, 818, 1635, 2358, 4699]"
3,ImpactPrincess,1955472014,0.63,242,642,1613,23179,55,28995,14.370118,...,17.975821,0.034098,0.398016,[0],"[0, 115, 577]",0,0,1,0.63,"[0, 3587]"
4,Damek0Masca,406301175,1.13,116,239,2318,2407,12,1058,1.038395,...,0.456428,0.005177,0.103106,[],[],None,4,0,0,[]


# Seeds

In [7]:
######## All Seeds 99 ########
seeds = network.copy()

record_num_network = network.describe().iloc[0,0]
record_num_network = int(record_num_network)
for i in range(record_num_network):
    record = network.loc[i,"generation"]
    if record != 0:
        seeds.drop([i],axis=0, inplace = True)
seeds.head()

,user,user_id,source_id,time_lapsed,followers_count,friends_count,seed_user_id,generation,time_since_seed
0,davelackie,100766356,NaN,0.00,143119,4643,1.007664e+08,0.0,0.0
4,Damek0Masca,406301175,NaN,1.13,116,239,4.063012e+08,0.0,0.0
21,Fashionista_com,15933910,NaN,17.00,2254291,825,1.593391e+07,0.0,0.0
25,LilEsBella,15098105,NaN,18.68,2038,367,1.509810e+07,0.0,0.0
34,tuffyhardy,2886562144,NaN,22.82,20,50,2.886562e+09,0.0,0.0


In [18]:
######## Effective Seeds 24 ########
effective_seeds= seeds.copy()

record_num_seeds = seeds.describe().iloc[0,0]
record_num_seeds = int(record_num_seeds)
for i in range(record_num_seeds):
    k=0
    user_index = seeds.index[i]
    user_id = seeds.loc[user_index,"user_id"]
    for j in range(record_num_network):
        source_id = network.loc[j,"source_id"]
        if user_id == source_id:
            k=k+1
            #print(k)
    if k == 0:
        effective_seeds.drop([user_index],axis=0, inplace = True)
effective_seeds.head()

,user,user_id,source_id,time_lapsed,followers_count,friends_count,seed_user_id,generation,time_since_seed
0,davelackie,100766356,NaN,0.00,143119,4643,1.007664e+08,0.0,0.0
21,Fashionista_com,15933910,NaN,17.00,2254291,825,1.593391e+07,0.0,0.0
25,LilEsBella,15098105,NaN,18.68,2038,367,1.509810e+07,0.0,0.0
81,ninatypewriter,171656520,NaN,28.13,10483,4108,1.716565e+08,0.0,0.0
416,TalkFilmSoc,2236860644,NaN,47.12,6281,2244,2.236861e+09,0.0,0.0


# Model_and_XData

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226984 entries, 0 to 56355
Data columns (total 69 columns):
Nw_averageNeighborDegree                      226984 non-null float64
Nw_averageNeighborDegree-1                    226984 non-null float64
Nw_averageNeighborDegree0                     226984 non-null float64
Nw_degreeCentrality                           226984 non-null float64
Nw_degreeCentrality-1                         226984 non-null float64
Nw_degreeCentrality0                          226984 non-null float64
Nw_eigenvectorCentrality                      226984 non-null float64
Nw_eigenvectorCentrality-1                    226984 non-null float64
Nw_eigenvectorCentrality0                     226984 non-null float64
Nw_inDegreeCentrality                         226984 non-null float64
Nw_inDegreeCentrality-1                       226984 non-null float64
Nw_inDegreeCentrality0                        226984 non-null float64
Nw_nNodes                                     2269

In [9]:
data.head(5)

,Nw_averageNeighborDegree,Nw_averageNeighborDegree-1,Nw_averageNeighborDegree0,Nw_degreeCentrality,Nw_degreeCentrality-1,Nw_degreeCentrality0,Nw_eigenvectorCentrality,Nw_eigenvectorCentrality-1,Nw_eigenvectorCentrality0,Nw_inDegreeCentrality,...,UsM_normalizedUserListedCount-1,UsM_normalizedUserListedCount0,UsM_normalizedUserStatusesCount,UsM_normalizedUserStatusesCount-1,UsM_normalizedUserStatusesCount0,UsM_statusesCount,UsM_statusesCount-1,UsM_statusesCount0,label,user_id
0,6.5,4.42623,4.42623,0.000599,0.045709,0.045709,0.000051,0.001645,0.001645,0.0002,...,0.185433,0.185433,12.334933,60.605747,60.605747,12853,181393,181393,1,3186545203
1,6.5,4.42623,4.42623,0.000599,0.045709,0.045709,0.000051,0.001645,0.001645,0.0002,...,0.185433,0.185433,12.334933,60.605747,60.605747,12853,181393,181393,1,3186545203
2,6.5,4.42623,4.42623,0.000599,0.045709,0.045709,0.000051,0.001645,0.001645,0.0002,...,0.185433,0.185433,12.334933,60.605747,60.605747,12853,181393,181393,1,3186545203
3,6.5,4.42623,4.42623,0.000599,0.045709,0.045709,0.000051,0.001645,0.001645,0.0002,...,0.185433,0.185433,12.334933,60.605747,60.605747,12853,181393,181393,1,3186545203
4,6.5,4.42623,4.42623,0.000599,0.045709,0.045709,0.000051,0.001645,0.001645,0.0002,...,0.185433,0.185433,12.334933,60.605747,60.605747,12853,181393,181393,1,3186545203


In [10]:
######## Model ########

def loadModel(self):
    read_file = "data/" + self.model_name + ".dat"
    self.model = joblib.load(read_file)
    return self.model

model = joblib.load(path+"/RandomForest_0_model.dat")

model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=10, verbose=0, warm_start=False)

In [11]:
######## Prediction ########
temp_X = data.drop(["label","UsM_favouritesCount","UsM_favouritesCount-1",
                    "UsM_listedCount","UsM_normalizedUserFollowersCount",
                    "UsM_normalizedUserFriendsCount","UsM_normalizedUserStatusesCount-1",
                    "UsM_statusesCount-1"],axis=1)

    


# infec = model.predict_proba(valid_X)
# infec
# infec[0:4]   #左闭右开 右大1左大0
# infected = []

# for line in infec:
#     if line[1]>0.5:
#         infected.append(1)
#     else:
#         infected.append(0)
# infected

# SIMULATION

In [12]:
# network_simulation = pd.DataFrame(columns=["user","user_id","source_candidates","infec_candidates",
#                                            "followers_count","friends_count","source_id","seed_user_id",
#                                            "SIR","generations","time_lapsed","time_since_seed"])
# network_simulation["user"] = users["user"]
# network_simulation["user_id"] = users["user_id"]
# network_simulation["source_candidates"] = users["source_candidates"]
# network_simulation["infec_candidates"] = users["infec_candidates"]
# network_simulation["followers_count"] = users["followers_count"]
# network_simulation["friends_count"] = users["friends_count"]

# network_simulation.head()

In [19]:
network_simulation = pd.DataFrame(columns=["user","user_id","source_candidates","infec_candidates",
                                           "followers_count","friends_count","source_id","seed_user_id",
                                           "SIR","generations","time_lapsed","time_since_seed"])
network_simulation["user"] = users["user"]
network_simulation["user_id"] = users["user_id"]
network_simulation["source_candidates"] = users["source_candidates"]
network_simulation["infec_candidates"] = users["infec_candidates"]
network_simulation["followers_count"] = users["followers_count"]
network_simulation["friends_count"] = users["friends_count"]

###sim###
time = 10
 

###seeds###
for i in range(len(seeds)):
    index = seeds.index[i]
    network_simulation.loc[index,"SIR"] = 1
    network_simulation.loc[index,"generations"] = 0
    network_simulation.loc[index,"source_id"] = network_simulation.loc[index,"user_id"]
    network_simulation.loc[index,"seed_user_id"] = network_simulation.loc[index,"user_id"]
    network_simulation.loc[index,"time_lapsed"] = seeds.loc[index,"time_lapsed"]
    network_simulation.loc[index,"time_since_seed"] = 0


###simulation###
for t in range(time):
    for i in range(record_num):
        if np.isnan(network_simulation.loc[i,"time_lapsed"]) == False:
            if round(60*network_simulation.loc[i,"time_lapsed"]) == t and network_simulation.loc[i,"SIR"] == 1:
                infec_candidates=network_simulation.loc[i,"infec_candidates"]
                for j in range(len(infec_candidates)):
                    infector = network_simulation.loc[i,"infec_candidates"][j]
                    user_id1 = network_simulation.loc[infector,"user_id"]
                    user_id2 = temp_X[temp_X['user_id'].isin([user_id1])]
                    if user_id2.empty: 
                        continue
                    user_id2["index"]=""
                    for tem in range(len(user_id2)):
                        
                        user_id2.iat[tem,61]=int(tem)
                        
                    if len(user_id2)<48:
                        if round(t/30)<48-len(user_id2):
                            useful_line =  user_id2[user_id2['index'].isin([0])]
                        else:
                            useful_line = user_id2[user_id2['index']].isin([round(t/30)+(48-len(user_id2))])
                    else:
                        useful_line =  user_id2[user_id2['index'].isin([round(t/30)])]
                    #print(useful_line)
                    #user_id2.loc[round(t/(60*30))]
                    valid1 = useful_line.drop(["user_id","index"],axis=1)
                    if valid1.empty:
                        print("empty input")
                        continue
                    infec = model.predict_proba(valid1)
                    if float(infec[:,1])>0.5 and network_simulation.loc[infector,"SIR"]!=2 and network_simulation.loc[infector,"SIR"]!=1:
                        network_simulation.loc[infector,"SIR"] = 1
                        network_simulation.loc[infector,"generations"] = network_simulation.loc[i,"generations"]+1
                        network_simulation.loc[infector,"source_id"] = network_simulation.loc[i,"user_id"] 
                        network_simulation.loc[infector,"seed_user_id"] = network_simulation.loc[i,"seed_user_id"] 
                        network_simulation.loc[infector,"time_lapsed"] = (t+1)/30
                network_simulation.loc[i,"SIR"]=2 


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:
network_simulation

,user,user_id,source_candidates,infec_candidates,followers_count,friends_count,source_id,seed_user_id,SIR,generations,time_lapsed,time_since_seed
0,davelackie,100766356,"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...","[1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",143119,4643,100766356,100766356,2,0,0,0
1,alexanderskhan,3186545203,[0],"[14, 4334]",753,3428,NaN,NaN,NaN,NaN,NaN,NaN
2,consiglierela,4134992843,"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...","[12, 32, 818, 1635, 2358, 4699]",2161,396,100766356,100766356,2,1,0.0333333,NaN
3,ImpactPrincess,1955472014,"[0, 115, 577]","[0, 3587]",242,642,NaN,NaN,NaN,NaN,NaN,NaN
4,Damek0Masca,406301175,[],[],116,239,406301175,406301175,1,0,1.13,0
5,panagiota_karag,2938214163,"[0, 20, 577, 1012, 1591, 1656, 1807, 1953, 320...","[0, 20, 306, 1591, 1953, 3203, 3810]",250,493,NaN,NaN,NaN,NaN,NaN,NaN
6,Gwen_UsBeauty,89805327,"[0, 21, 748, 1912, 1929]","[0, 20, 1656, 1912, 2504, 2811, 4768, 4930]",39697,1924,100766356,100766356,2,1,0.0333333,NaN
7,mascott04,342931336,[0],"[0, 306]",672,353,100766356,100766356,2,1,0.0333333,NaN
8,FazeyPie,27220084,[0],[],275,440,NaN,NaN,NaN,NaN,NaN,NaN
9,RissaRunsThis,97360017,[0],[],1911,506,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
users

NameError: name 'users' is not defined